In [ ]:
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(
    temperature=0.1
)

In [ ]:
from langchain.schema import HumanMessage, SystemMessage, AIMessage

messages = [
    SystemMessage(content="You are a a geography expert. And you only reply in Itailian."),
    AIMessage(content="Ciao, mi chiamo Paolo!"),
    HumanMessage(content="What is the distance between Rome and Naples? Also, what is your name?"),
]

chat.predict_messages(messages)



In [ ]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate

template = PromptTemplate.from_template("Whate is the distance between {country_a} and {country_b}?")
prompt = template.format(country_a="Italy", country_b="France")

chat.predict(prompt)



In [ ]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate

template = ChatPromptTemplate.from_messages([
    ("system", "You are a a geography expert. And you only reply in {language}."),
    ("ai", "Ciao, mi chiamo {name}!"),
    ("human", "What is the distance between {country_a} and {country_b}? Also, what is your name?"),
])

prompt = template.format_messages(language="Greek", country_a="Italy", country_b="France", name="Paolo")


chat.predict_messages(prompt)


In [ ]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    def parse(self, text: str) -> str:
        items = text.strip().split(",")
        return [item.strip() for item in items]

parser = CommaOutputParser()
parser.parse("Hello, world!")




In [ ]:
from langchain.prompts import ChatPromptTemplate

template = ChatPromptTemplate.from_messages([
    ("system", "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items} in lower case. Do NOT reply with anything else."),
    ("human", "{question}"),
])

prompt = template.format_messages(question="What are the colors??", max_items=10)
result = chat.predict_messages(prompt)

parser = CommaOutputParser()
parser.parse(result.content)


In [ ]:
chain = template | chat | CommaOutputParser()

chain.invoke({"question": "What are the colors?", "max_items": 10})


In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

writing_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are specialized in writing haikus about programming languages. Always generate responses in the form of a haiku (5-7-5 syllable structure) and ensure each haiku centers around themes of programming, coding languages, or software development. Be concise, creative, and poetic."),
    ("human", "{language}"),
])

writing_chain = writing_prompt | chat 

reading_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are specialized in explaining haikus. When given a haiku, analyze its structure (syllable pattern, imagery, seasonal or thematic elements) and explain its meaning in a clear, thoughtful way in Korean. Focus on uncovering the poetic devices, cultural context, and emotional nuance. Be concise, insightful, and accessible to readers."),
    ("human", "{haiku}"),
])

reading_chain = reading_prompt | chat

final_chain = {"haiku": writing_chain} | reading_chain

final_chain.invoke({"language": "nodejs"})



In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import FewShotPromptTemplate
from langchain.prompts import PromptTemplate

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """
    },
]

example_prompt = PromptTemplate.from_template("Human: {question}\nAI: {answer}")

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"]
)

chain = prompt | chat

chain.invoke({"country": "Korea"})

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import FewShotChatMessagePromptTemplate
from langchain.prompts import ChatPromptTemplate

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

examples = [
    {
        "country": "France",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """
    },
    {
        "country": "Italy",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """
    },
    {
        "country": "Greece",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """
    },
]

example_prompt = ChatPromptTemplate.from_messages([
    ("human", "What do you know about {country}?"),
    ("ai", "{answer}")
])

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a a geography expert, you give short and concise answers."),
    example_prompt,
    ("human", "What do you know about {country}?"),
])

chain = final_prompt | chat

chain.invoke({"country": "Japan"})

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import FewShotChatMessagePromptTemplate
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """
    },
]

example_prompt = PromptTemplate.from_template("Human: {question}\nAI: {answer}")

example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=180
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"]
)

prompt.format(country="Brazil")

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import FewShotChatMessagePromptTemplate
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.example_selector.base import BaseExampleSelector

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """
    },
]

class RandomExampleSelector(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples

    def add_example(self, example):
        self.examples.append(example)

    def select_examples(self, input_variables):
        from random import choice
        return [choice(self.examples)]



example_prompt = PromptTemplate.from_template("Human: {question}\nAI: {answer}")

example_selector = RandomExampleSelector(
    examples=examples
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"]
)

prompt.format(country="Brazil")

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import load_prompt

prompt = load_prompt("prompt.yaml")

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

prompt.format(country="Brazil")

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.prompts.pipeline import PipelinePromptTemplate

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}
"""
)

example = PromptTemplate.from_template(
    """
    This is an example of how you talk:

    Human: {example_question}
    You: {example_answer}
"""
)

start = PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You:
"""
)

final = PromptTemplate.from_template(
    """
    {intro}
                                     
    {example}
                              
    {start}
"""
)

prompts = [
    ("intro", intro),
    ("example", example),
    ("start", start),
]


full_prompt = PipelinePromptTemplate(
    final_prompt=final,
    pipeline_prompts=prompts,
)


chain = full_prompt | chat

chain.invoke(
    {
        "character": "Pirate",
        "example_question": "What is your location?",
        "example_answer": "Arrrrg! That is a secret!! Arg arg!!",
        "question": "What is your fav food?",
    }
)

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache, SQLiteCache

set_llm_cache(SQLiteCache())
set_debug(True)

chat = ChatOpenAI(
    temperature=0.1,
)

chat.predict("How do you make itailian pasta?")

In [ ]:
chat.predict("How do you make itailian pasta?")

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback

chat = ChatOpenAI(
    temperature=0.1,
)

with get_openai_callback() as usage:
    a = chat.predict("What is the recipe for soju?")
    b = chat.predict("What is the recipe for bread?")
    print(a, b, "\n")
    print(usage)



In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.llms.openai import OpenAI
from langchain.llms.loading import load_llm

chat = load_llm("model.json")
chat

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import FewShotChatMessagePromptTemplate
from langchain.prompts import ChatPromptTemplate

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

examples = [
    {
        "movie": "The Dark Knight",
        "answer": """
        Here is what I know about this movie:
        Director: Christopher Nolan
        Main Cast: Christian Bale, Heath Ledger, Aaron Eckhart, Michael Caine
        Budget: $185 million
        Box Office Revenue: $1.005 billion
        Genre: Action, Crime, Drama
        Synopsis: Batman faces his greatest psychological and physical tests when the Joker emerges as a force of chaos to test whether Batman can make the ultimate sacrifice to save Gotham City.
        """
    },
    {
        "movie": "Inception",
        "answer": """
        Here is what I know about this movie:
        Director: Christopher Nolan
        Main Cast: Leonardo DiCaprio, Marion Cotillard, Tom Hardy, Ellen Page
        Budget: $160 million
        Box Office Revenue: $836.8 million
        Genre: Action, Sci-Fi, Thriller
        Synopsis: A skilled thief who enters people's dreams to steal secrets is given a final job: instead of stealing an idea, he must plant one deep within a target's subconscious.
        """
    },
    {
        "movie": "Titanic",
        "answer": """
        Here is what I know about this movie:
        Director: James Cameron
        Main Cast: Leonardo DiCaprio, Kate Winslet, Billy Zane, Gloria Stuart
        Budget: $200 million
        Box Office Revenue: $2.257 billion
        Genre: Romance, Drama
        Synopsis: A young aristocrat falls in love with a kind but poor artist aboard the luxurious, ill-fated R.M.S. Titanic during its maiden voyage in 1912.
        """
    },
    {
        "movie": "Parasite",
        "answer": """
        Here is what I know about this movie:
        Director: Bong Joon-ho
        Main Cast: Song Kang-ho, Lee Sun-kyun, Cho Yeo-jeong, Choi Woo-shik
        Budget: $11.4 million
        Box Office Revenue: $258.8 million
        Genre: Comedy, Drama, Thriller
        Synopsis: A poor family schemes to become employed by a wealthy family by infiltrating their household and posing as unrelated, highly qualified individuals.
        """
    }
]

example_prompt = ChatPromptTemplate.from_messages([
    ("human", "What do you know about {movie}?"),
    ("ai", "{answer}")
])

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a movie expert who provides detailed information about films. Always include director, main cast, budget, box office revenue, genre, and synopsis in your responses using the exact same format."),
    few_shot_prompt,
    ("human", "What do you know about {movie}?"),
])

chain = final_prompt | chat

chain.invoke({"movie": "Les miserables"})

### Assignment 3

In [2]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import FewShotChatMessagePromptTemplate
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True,
)


examples = [
    {
        "movie": "Top Gun",
        "answer": """
        🛩️👨‍✈️🔥
        """
    },
    {
        "movie": "The Godfather",
        "answer": """
        👨‍👨‍👦🔫🍝
        """
    },
]


example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{movie}"),
    ("ai", "{answer}")
])

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a movie expert who provides three emojis to represent the movie."),
    few_shot_prompt,
    ("human", "{movie}?"),
])

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a movie expert who provides three emojis to represent the movie."),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

def load_memory(_):
    return memory.load_memory_variables({})["history"]

chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm

def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result)

invoke_chain("Parasite")

invoke_chain("Titanic")

content='🏠💰🔪'
content='🚢💑❄️'


In [3]:
invoke_chain("What is the movie I asked first?")

content='🏠💰🔪'
